# PulseAD Tuning Guide

Learn how to configure thresholds for optimal anomaly detection.

**Topics:**
- Understanding dual thresholds
- Custom threshold configuration
- Per-dimension overrides
- Sensitivity analysis

In [ ]:
import sys
sys.path.append('../..')

from gradientcast import GradientCastPulseAD, ThresholdConfig
from utils.synthetic_data import generate_timestamps
import matplotlib.pyplot as plt
import numpy as np

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

ad = GradientCastPulseAD(api_key=GRADIENTCAST_API_KEY)

---
## Understanding Dual Thresholds

PulseAD uses two thresholds to detect anomalies:

| Threshold | Default | Purpose |
|-----------|---------|--------|
| `percentage_threshold` | 15% | Minimum deviation to flag |
| `minimum_value_threshold` | 100,000 | Filter out low-volume noise |

An anomaly is only flagged when **both** conditions are met.

In [ ]:
# Sample data with various deviation levels
def create_test_data(values):
    timestamps = generate_timestamps(len(values), freq='H')
    return [{"timestamp": ts, "value": int(v)} for ts, v in zip(timestamps, values)]

# Normal range around 1.5M, with deviations at different levels
test_values = [
    1500000, 1510000, 1490000, 1520000, 1505000,  # Normal (±2%)
    1350000,  # 10% drop
    1200000,  # 20% drop
    900000,   # 40% drop
]

data = {"metric": create_test_data(test_values)}

In [ ]:
# Default thresholds (15%)
result_default = ad.detect(data)

print("Default Thresholds (15%):")
print(f"  Anomalies found: {len(result_default.anomalies)}")
for a in result_default.anomalies:
    print(f"  - Value {a.actual_value:,}: {a.percent_delta} deviation")

---
## Custom Threshold Configuration

Use `ThresholdConfig` to customize detection sensitivity.

In [ ]:
# Stricter threshold (10%)
strict_config = ThresholdConfig(
    default_percentage=0.10,     # 10% threshold
    default_minimum=100000       # Same minimum
)

result_strict = ad.detect(data, threshold_config=strict_config)

print("Strict Thresholds (10%):")
print(f"  Anomalies found: {len(result_strict.anomalies)}")
for a in result_strict.anomalies:
    print(f"  - Value {a.actual_value:,}: {a.percent_delta} deviation")

In [ ]:
# Relaxed threshold (25%)
relaxed_config = ThresholdConfig(
    default_percentage=0.25,     # 25% threshold
    default_minimum=100000
)

result_relaxed = ad.detect(data, threshold_config=relaxed_config)

print("Relaxed Thresholds (25%):")
print(f"  Anomalies found: {len(result_relaxed.anomalies)}")
for a in result_relaxed.anomalies:
    print(f"  - Value {a.actual_value:,}: {a.percent_delta} deviation")

---
## Per-Dimension Overrides

Different metrics may need different thresholds.

In [ ]:
# Multi-dimensional data
multi_data = {
    "critical_metric": create_test_data([1500000, 1510000, 1350000]),  # 10% drop
    "secondary_metric": create_test_data([500000, 510000, 400000]),    # 20% drop
}

# Different thresholds per dimension
custom_config = ThresholdConfig(
    default_percentage=0.20,      # Default: 20%
    default_minimum=100000,
    per_dimension_overrides={
        "critical_metric": {
            "percentage_threshold": 0.05,  # Critical: only 5% tolerance
            "minimum_value_threshold": 1000000
        }
        # secondary_metric uses default (20%)
    }
)

result = ad.detect(multi_data, threshold_config=custom_config)

print("Per-Dimension Thresholds:")
for a in result.anomalies:
    print(f"  {a.dimension}: {a.percent_delta} deviation (threshold: {a.threshold})")

---
## Sensitivity Analysis

Visualize how different thresholds affect detection.

In [ ]:
# Test range of thresholds
thresholds = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
anomaly_counts = []

for pct in thresholds:
    config = ThresholdConfig(default_percentage=pct, default_minimum=100000)
    result = ad.detect(data, threshold_config=config)
    anomaly_counts.append(len(result.anomalies))

# Plot
plt.figure(figsize=(10, 5))
plt.bar([f"{int(t*100)}%" for t in thresholds], anomaly_counts, color='steelblue')
plt.xlabel('Percentage Threshold')
plt.ylabel('Anomalies Detected')
plt.title('Sensitivity Analysis: Threshold vs Detection Count')
plt.grid(alpha=0.3, axis='y')
plt.show()

print("\nRecommendation: Choose threshold based on acceptable false positive rate")

---
## Minimum Value Threshold

The minimum value threshold filters out noise on low-volume metrics.

In [ ]:
# Low volume data with large percentage swings
low_volume_data = {
    "small_metric": create_test_data([1000, 1100, 500, 1050])  # 50% drop, but low volume
}

# High minimum threshold (filters out low volume)
high_min_config = ThresholdConfig(
    default_percentage=0.15,
    default_minimum=10000  # Requires 10K minimum
)

result = ad.detect(low_volume_data, threshold_config=high_min_config)
print(f"With minimum=10000: {len(result.anomalies)} anomalies (filtered out low volume)")

# Low minimum threshold
low_min_config = ThresholdConfig(
    default_percentage=0.15,
    default_minimum=100  # Low minimum
)

result = ad.detect(low_volume_data, threshold_config=low_min_config)
print(f"With minimum=100: {len(result.anomalies)} anomalies (detected low volume)")

---
## Best Practices

### Choosing Percentage Threshold

| Scenario | Recommended | Rationale |
|----------|-------------|----------|
| High-stakes metrics | 5-10% | Catch small deviations early |
| Standard monitoring | 15-20% | Balance sensitivity/noise |
| Volatile metrics | 25-30% | Reduce false positives |

### Choosing Minimum Value

| Volume Level | Recommended | Example |
|--------------|-------------|--------|
| High volume | 1,000,000+ | User counts, page views |
| Medium volume | 100,000 | Daily active users |
| Low volume | 1,000-10,000 | Transactions, signups |

**Next:** [Real-time Simulation](03_simulation.ipynb)